In [1]:
#Initialize dependencies
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorboard as tb
BASE_FOLDER = '/Users/oblivisheee/Documents/GitHub/novelsdreamer-ru-t4m/'
os.chdir(BASE_FOLDER)
from modules.transformer_custom import Transformer
from modules.regularization import RegularizedDenseLayer
from modules.create_config import transformer_config, trainer_config, metrics_config
from modules.tf_text_postprocess import TextPostprocessor
from modules.datagen import DataGenerator

# Config training

In [2]:
# Configure the Transformer layer
config = transformer_config()
config_train = trainer_config()
config_metrics = metrics_config()

embedding_layer = tf.keras.layers.Embedding(config["input_vocab_size"], config["d_model"])

model = Transformer(config["num_layers"], config["d_model"], config["num_heads"], config["dff"],
                          config["input_vocab_size"], config["input_vocab_size"], pe_input=config["maximum_position_encoding"],
                          pe_target=config["maximum_position_encoding"], rate=config["dropout_rate"], embedding=embedding_layer)

regularized_layer = RegularizedDenseLayer(config["d_model"])

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=config_train['loss_reduction'])
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=config_train['learning_rate'])

metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name=config_metrics['accuracy_set']), 
           tf.keras.metrics.MeanSquaredError(name=config_metrics['mean_sq_error']), 
           tf.keras.metrics.Precision(thresholds=0.5, name=config_metrics['precision']),]

model.compile(optimizer=optimizer, loss=loss_object, metrics=metrics)



In [3]:
MAIN_DATASET_DIR = 'dataset'
TRAIN_DATASET_DIR = os.path.join(MAIN_DATASET_DIR, 'train')
VALID_DATASET_DIR = os.path.join(MAIN_DATASET_DIR, 'valid')

datagen = DataGenerator(TRAIN_DATASET_DIR, VALID_DATASET_DIR)
(train_english, train_russian), (valid_english, valid_russian) = datagen.generate()

{'russian': [<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>], 'english': [<tf.Tensor: shape=(2,), dtype=float32, numpy=array([3., 4.], dtype=float32)>]}
{'russian': [<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>], 'english': [<tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.], dtype=float32)>]}
Train data info: 2 classes, 2 samples
Valid data info: 2 classes, 2 samples


# Start training

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
epochs = config_train['epoch_train']
model.fit_model(train_english, 
                         train_russian,
                         valid_english,
                         valid_russian,
                         epochs=epochs,
                         save_model_each_epoch=True,
                         model_name='novelsdreamer-ru-t4m')

  0%|          | 0/50 [00:00<?, ?it/s]

INFO:tensorflow:Epoch 1 Loss 10.8152 Validation Loss 10.6063
INFO:tensorflow:Epoch 1 finished.


  2%|▏         | 1/50 [00:14<11:49, 14.49s/it]

INFO:tensorflow:Epoch 2 Loss 10.5868 Validation Loss 10.3131
INFO:tensorflow:Epoch 2 finished.


  4%|▍         | 2/50 [00:25<10:11, 12.73s/it]


KeyboardInterrupt: 

In [ ]:
epochs = config_train['epoch_train']
train_dataset = tf.data.Dataset.from_tensor_slices((train_english.tolist(), train_russian.tolist()))
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_english.tolist(), valid_russian.tolist()))

model.fit(train_dataset, 
          validation_data=valid_dataset,
          epochs=epochs,
          verbose='auto',
          shuffle=True, workers=4, validation_freq=10)
